<a href="https://colab.research.google.com/github/NazeerSiddique-dev/Nazeer-demo/blob/main/dualBranch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle -q

from google.colab import files
print("Please upload your kaggle.json file:")
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
print("\n✅ Kaggle API configured successfully!")

Please upload your kaggle.json file:


Saving kaggle.json to kaggle.json

✅ Kaggle API configured successfully!


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!kaggle datasets download -d toygarr/camus-dataset

!unzip -q camus-dataset.zip -d /content/camus_dataset

Dataset URL: https://www.kaggle.com/datasets/toygarr/camus-dataset
License(s): unknown
 85% 417M/493M [00:03<00:00, 124MB/s] 
100% 493M/493M [00:03<00:00, 134MB/s]


In [ ]:
import os, h5py

h5_path = "/content/camus_dataset/image_dataset.hdf5"  # adjust if different
print("Exists:", os.path.exists(h5_path), h5_path)

with h5py.File(h5_path, "r") as f:
    def walk(name, obj):
        if isinstance(obj, h5py.Dataset):
            print("DATASET:", name, "shape:", obj.shape, "dtype:", obj.dtype)
        else:
            print("GROUP:", name)
    f.visititems(walk)


Exists: True /content/camus_dataset/image_dataset.hdf5
DATASET: train 2ch frames shape: (900, 384, 384, 1) dtype: float32
DATASET: train 2ch masks shape: (900, 384, 384, 1) dtype: int32
DATASET: train 4ch frames shape: (900, 384, 384, 1) dtype: float32
DATASET: train 4ch masks shape: (900, 384, 384, 1) dtype: int32


In [ ]:
import h5py, csv, random
from pathlib import Path

SPLIT = {"train": 0.8, "val": 0.1, "test": 0.1}
SEED = 42

with h5py.File(h5_path, "r") as f:
    k2f = "train 2ch frames"; k4f = "train 4ch frames"
    n2, n4 = len(f[k2f]), len(f[k4f])
samples = [("2ch", i) for i in range(n2)] + [("4ch", i) for i in range(n4)]
random.Random(SEED).shuffle(samples)

n = len(samples)
n_train = int(SPLIT["train"] * n); n_val = int(SPLIT["val"] * n)
train_split = samples[:n_train]
val_split   = samples[n_train:n_train+n_val]
test_split  = samples[n_train+n_val:]

def write_split_csv(rows, path):
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", newline="") as f:
        w = csv.writer(f); w.writerow(["view","idx"]); w.writerows(rows)

split_dir = Path("/content/camus_splits_csv")
write_split_csv(train_split, split_dir/"train.csv")
write_split_csv(val_split,   split_dir/"val.csv")
write_split_csv(test_split,  split_dir/"test.csv")
print("Wrote splits to", split_dir)


Wrote splits to /content/camus_splits_csv


In [ ]:
from torch.utils.data import Dataset
import numpy as np, torch, cv2, h5py
import albumentations as A
from albumentations.pytorch import ToTensorV2
import csv

class GlobalLocalCamusHDF5(Dataset):
    def __init__(self, h5_path, split_csv,
                 global_size=256, local_base=384, local_out=256,
                 roi_class=1, p_roi_center=0.7, augment=True):
        self.h5_path = h5_path
        self.global_size = global_size
        self.local_base = local_base
        self.local_out = local_out
        self.roi_class = roi_class
        self.p_roi_center = p_roi_center
        self.augment = augment
        self.items = []
        with open(split_csv, "r") as f:
            r = csv.DictReader(f)
            for row in r:
                self.items.append((row["view"], int(row["idx"])))
        # Albumentations
        self.tf_global = A.Compose([
            A.Resize(global_size, global_size),
            A.Normalize(mean=0.5, std=0.5),
            ToTensorV2()
        ])
        self.tf_local_norm = A.Compose([
            A.Normalize(mean=0.5, std=0.5),
            ToTensorV2()
        ])
        self.tf_up = A.Resize(local_base, local_base)
        self.tf_aug = A.Compose([
            A.Rotate(limit=25, p=0.7),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.4),
        ])

    def __len__(self): return len(self.items)

    @staticmethod
    def _centroid(mask, cls):
        m = (mask == cls).astype(np.uint8)
        if m.sum() == 0: return None
        ys, xs = np.where(m)
        return int(xs.mean()), int(ys.mean())

    @staticmethod
    def _crop_around(arr, cx, cy, size):
        H, W = arr.shape[:2]
        x0 = max(0, cx - size // 2); y0 = max(0, cy - size // 2)
        x1 = min(W, x0 + size);      y1 = min(H, y0 + size)
        x0 = max(0, x1 - size);      y0 = max(0, y1 - size)
        return arr[y0:y1, x0:x1]

    def __getitem__(self, idx):
        view, i = self.items[idx]
        with h5py.File(self.h5_path, "r") as f:
            if view == "2ch":
                img = f["train 2ch frames"][i][..., 0].astype(np.float32)
                msk = f["train 2ch masks"][i][..., 0].astype(np.int32)
            else:
                img = f["train 4ch frames"][i][..., 0].astype(np.float32)
                msk = f["train 4ch masks"][i][..., 0].astype(np.int32)

        if self.augment:
            aug = self.tf_aug(image=img, mask=msk)
            img, msk = aug["image"], aug["mask"]

        # global transform → tensors
        g = self.tf_global(image=img, mask=msk)
        g_img = g["image"]  # torch [1,S,S]

        # local: upsample canvas (numpy), crop around ROI, then tensorize
        up = self.tf_up(image=img, mask=msk)
        u_img, u_msk = up["image"], up["mask"]
        use_roi = (np.random.rand() < self.p_roi_center)
        c = self._centroid(u_msk, self.roi_class) if use_roi else None
        if c is None:
            c = (u_img.shape[1] // 2, u_img.shape[0] // 2)
        cx, cy = c
        l_img_np = self._crop_around(u_img, cx, cy, self.local_out)
        l_msk_np = self._crop_around(u_msk, cx, cy, self.local_out)
        l = self.tf_local_norm(image=l_img_np, mask=l_msk_np)
        l_img = l["image"]          # torch [1,S,S]
        l_msk = l["mask"].long()    # torch [S,S] int64 for CE

        return (g_img, l_img), l_msk


In [ ]:
from torch.utils.data import DataLoader

train_csv = "/content/camus_splits_csv/train.csv"
val_csv   = "/content/camus_splits_csv/val.csv"

train_ds = GlobalLocalCamusHDF5(h5_path, train_csv, global_size=256, local_base=384, local_out=256,
                                roi_class=1, p_roi_center=0.7, augment=True)
val_ds   = GlobalLocalCamusHDF5(h5_path, val_csv,   global_size=256, local_base=384, local_out=256,
                                roi_class=1, p_roi_center=1.0, augment=False)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True,  num_workers=2, pin_memory=True, drop_last=True)
val_loader   = DataLoader(val_ds,   batch_size=8, shuffle=False, num_workers=2, pin_memory=True)
print("Train:", len(train_ds), "Val:", len(val_ds))


Train: 1440 Val: 180


In [ ]:
import torch.nn as nn
from torchvision.models.segmentation import deeplabv3_resnet50

class DLv3Backbone1ch(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        self.adapter = nn.Conv2d(1, 3, kernel_size=1)
        self.model = deeplabv3_resnet50(weights_backbone=None, num_classes=num_classes)
    def forward(self, x):
        x = self.adapter(x)
        return self.model(x)["out"]  # BxCxHxW

class LogitMixer(nn.Module):
    def __init__(self, in_ch, out_ch, mid=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, mid, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid, out_ch, 1),
        )
    def forward(self, g_logits, l_logits):
        return self.net(torch.cat([g_logits, l_logits], dim=1))

class GlobalLocalSeg(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        self.global_branch = DLv3Backbone1ch(num_classes)
        self.local_branch  = DLv3Backbone1ch(num_classes)
        self.mixer = LogitMixer(2 * num_classes, num_classes)
    def forward(self, xg, xl):
        g = self.global_branch(xg)
        l = self.local_branch(xl)
        fused = self.mixer(g, l)
        return fused, g, l


In [ ]:
!pip -q install torchmetrics
import torch, torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchmetrics.classification import MulticlassJaccardIndex
from torchmetrics.segmentation import DiceScore
from tqdm.auto import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GlobalLocalSeg(num_classes=4).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3,
                              threshold=1e-4, threshold_mode='rel', min_lr=1e-6)

def build_metrics():
    miou = MulticlassJaccardIndex(num_classes=4, average='macro').to(device)
    dice = DiceScore(num_classes=4, average='macro').to(device)
    return miou, dice

history = {"train_loss": [], "val_loss": [], "train_dice": [], "val_dice": [], "train_miou": [], "val_miou": []}
best_val, bad_epochs, early_patience, min_delta = float('inf'), 0, 8, 1e-4
max_epochs = 50

for epoch in range(max_epochs):
    model.train()
    tr_loss = 0.0
    tr_miou_metric, tr_dice_metric = build_metrics()
    for (g_img, l_img), y in tqdm(train_loader, desc=f"Train {epoch+1}/{max_epochs}"):
        g_img, l_img, y = g_img.to(device), l_img.to(device), y.to(device)
        logits, _, _ = model(g_img, l_img)
        loss = criterion(logits, y)
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        tr_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        tr_miou_metric.update(preds, y)
        tr_dice_metric.update(F.one_hot(preds, 4).permute(0,3,1,2).float(),
                              F.one_hot(y,    4).permute(0,3,1,2).float())
    tr_loss /= max(1, len(train_loader))
    tr_miou = tr_miou_metric.compute().item()
    tr_dice = tr_dice_metric.compute().item()

    model.eval()
    va_loss = 0.0
    va_miou_metric, va_dice_metric = build_metrics()
    with torch.no_grad():
        for (g_img, l_img), y in tqdm(val_loader, desc="Val", leave=False):
            g_img, l_img, y = g_img.to(device), l_img.to(device), y.to(device)
            logits, _, _ = model(g_img, l_img)
            va_loss += criterion(logits, y).item()
            preds = torch.argmax(logits, dim=1)
            va_miou_metric.update(preds, y)
            va_dice_metric.update(F.one_hot(preds, 4).permute(0,3,1,2).float(),
                                  F.one_hot(y,    4).permute(0,3,1,2).float())
    va_loss /= max(1, len(val_loader))
    va_miou = va_miou_metric.compute().item()
    va_dice = va_dice_metric.compute().item()

    prev_lr = optimizer.param_groups[0]['lr']
    scheduler.step(va_loss)
    curr_lr = optimizer.param_groups[0]['lr']
    if curr_lr < prev_lr:
        print(f"LR reduced: {prev_lr:.2e} -> {curr_lr:.2e}")

    history["train_loss"].append(tr_loss); history["val_loss"].append(va_loss)
    history["train_dice"].append(tr_dice); history["val_dice"].append(va_dice)
    history["train_miou"].append(tr_miou); history["val_miou"].append(va_miou)

    torch.save({"epoch": epoch, "model": model.state_dict(), "optim": optimizer.state_dict(),
                "val_loss": va_loss, "history": history}, "lastcheckpoint_gl.pth")

    if (best_val - va_loss) > min_delta:
        best_val = va_loss; bad_epochs = 0
        torch.save({"epoch": epoch, "model": model.state_dict(), "optim": optimizer.state_dict(),
                    "val_loss": best_val, "history": history}, "bestcheckpoint_gl.pth")
        print(f"Epoch {epoch+1}: loss tr {tr_loss:.4f} va {va_loss:.4f} | Dice tr {tr_dice:.4f} va {va_dice:.4f} | mIoU tr {tr_miou:.4f} va {va_miou:.4f} -> new best")
    else:
        bad_epochs += 1
        print(f"Epoch {epoch+1}: loss tr {tr_loss:.4f} va {va_loss:.4f} | Dice tr {tr_dice:.4f} va {va_dice:.4f} | mIoU tr {tr_miou:.4f} va {va_miou:.4f} (no improv {bad_epochs}/{early_patience})")
        if bad_epochs >= early_patience:
            print("Early stopping."); break


Train 1/50:   0%|          | 0/180 [00:00<?, ?it/s]

Val:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch 1: loss tr 0.4763 va 0.8039 | Dice tr 0.7415 va 0.5530 | mIoU tr 0.6169 va 0.4290 -> new best


Train 2/50:   0%|          | 0/180 [00:00<?, ?it/s]

Val:   0%|          | 0/23 [00:00<?, ?it/s]

...

Epoch 31: loss tr 0.1275 va 0.1593 | Dice tr 0.9251 va 0.9105 | mIoU tr 0.8759 va 0.8532 (no improv 1/8)


Train 32/50:   0%|          | 0/180 [00:00<?, ?it/s]

Val:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch 32: loss tr 0.1262 va 0.1490 | Dice tr 0.9242 va 0.9144 | mIoU tr 0.8770 va 0.8589 (no improv 2/8)


Train 33/50:   0%|          | 0/180 [00:00<?, ?it/s]